In [ ]:
# M08: 

# sales
# https://caenergy-my.sharepoint.com/:x:/g/personal/jesten_ruiz_energy_ca_gov/ETcZua8EH95EkQEOA0a3OZcBFqAQlmGkfitkEA3KnFapWg?email=Fernando.Greve%40Energy.ca.gov&wdOrigin=TEAMS-MAGLEV.p2p_ns.rwc&wdExp=TEAMS-TREATMENT&wdhostclicktime=1736878801692&web=1&clickparams=eyJBcHBOYW1lIjoiVGVhbXMtRGVza3RvcCIsIkFwcFZlcnNpb24iOiI0OS8yNDEyMDEwMDIxMyIsIkhhc0ZlZGVyYXRlZFVzZXIiOmZhbHNlfQ%3D%3D
# https://caenergy-my.sharepoint.com/:x:/g/personal/jesten_ruiz_energy_ca_gov/EZ5jsGmt02BDqpl4Vsupu4gBiKP1g9FcLwppVhztSZ98qg?email=Fernando.Greve%40Energy.ca.gov&wdOrigin=TEAMS-MAGLEV.p2p_ns.rwc&wdExp=TEAMS-TREATMENT&wdhostclicktime=1736878863838&web=1&clickparams=eyJBcHBOYW1lIjoiVGVhbXMtRGVza3RvcCIsIkFwcFZlcnNpb24iOiI0OS8yNDEyMDEwMDIxMyIsIkhhc0ZlZGVyYXRlZFVzZXIiOmZhbHNlfQ%3D%3D


In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [7]:
wd = os.getcwd()
rawdata_path = os.path.join(wd, "rawdata")
data_path = os.path.join(wd, "data")

In [8]:
def df_to_rawdata(df, filename):
    """Save df into rawdata folder."""    
    # df['Date'] = df.index
    # df = df.set_index("Date")    
    df.to_csv(os.path.join(rawdata_path, filename + ".csv"), index=False)

In [9]:
def rawdata_to_df(filename):
    # df['Date'] = df.index
    # df = df.set_index("Date")
    df = pd.read_csv(os.path.join(rawdata_path, filename + ".csv"))    
    return df

In [10]:
def df_to_data(df, filename):
    """Save df into data folder."""    
    # df['Date'] = df.index
    # df = df.set_index("Date")    
    df.to_csv(os.path.join(data_path, filename + ".csv"), index=False)

In [3]:
# grab the datasources we have access to
pyodbc.dataSources()

{'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)'}

In [4]:
# define components of our connection string
driver = '{Microsoft Access Driver (*.mdb, *.accdb)}'
# filepath = r'C:\Users\Alex\OneDrive\Career - Work Items\Petco\Financial_Analyst\Petco_Financial_Data.accdb'
filepath = r'P:\ADMIN\PIIRA\NewPIIRA2004.mdb'

In [5]:
# create a connection to the database
conn = pyodbc.connect(driver = driver, dbq = filepath, autocommit = True)
cursor = conn.cursor()

In [6]:
# grab all the tables
tables_list = list(cursor.tables())
tables_list

[('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysAccessStorage', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysAccessXML', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysIMEXColumns', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysIMEXSpecs', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysNameMap', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysNavPaneGroupCategories', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysNavPaneGroups', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysNavPaneGroupToObjects', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysNavPaneObjectIDs', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\NewPIIRA2004.mdb', None, 'MSysObjects', 'SYSTEM TABLE', None),
 ('P:\\ADMIN\\PIIRA\\

In [41]:
tablas = [list(row)[2] for row in tables_list if list(row)[-2]=='TABLE']
[row for row in tablas if row.find('800')>-1]

['tbl_PIIRAE800', 'tbl_PIIRAE800_bckup111130']

In [11]:
def table_to_rawdata(table_name):
    """Save table into rawdata folder."""    
    
    # define query
    query = "SELECT * FROM {}".format(table_name)
    
    # execute the query
    cursor.execute(query)
    
    df = pd.read_sql(query, conn)
    df_to_rawdata(df, table_name)

In [12]:
table_name = 'tbl_PIIRAWM08'
table_to_rawdata(table_name)

In [13]:
table_name = 'tbl_PIIRAE810_NEW'
table_to_rawdata(table_name)

In [22]:
table_name = 'tbl_PIIRA_CECM810'
table_to_rawdata(table_name)

In [25]:
table_name = 'tbl_PIIRA782B'
table_to_rawdata(table_name)

In [42]:
table_name = 'tbl_PIIRAE800'
table_to_rawdata(table_name)

# 800

In [43]:
filename = "tbl_PIIRAE800"
df = rawdata_to_df(filename)
df

,Date,Company_ID,Refinery_ID,Product_Code,Production,Stocks
0,2023-10-27,9232497.0,621001,134,0.0,197.0
1,2023-10-27,9232497.0,621001,213,0.0,749.0
2,2023-10-27,9232497.0,621001,217,0.0,244.0
3,2023-10-27,9232497.0,621001,218,0.0,505.0
4,2023-10-27,9232497.0,621001,461,0.0,0.0
...,...,...,...,...,...,...
421344,2025-01-03,66647066.0,897101,477,0.0,0.0
421345,2025-01-03,66647066.0,897101,478,99.0,103.0
421346,2025-01-03,66647066.0,897101,462,0.0,0.0
421347,2025-01-03,66647066.0,897101,205,0.0,0.0


# 782

In [26]:
filename = "tbl_PIIRA782B"
df = rawdata_to_df(filename)
df

,Year,Month,CompanyID,FuelType,SaleType,PointofSale,ProductCode,Volume,Price,F10
0,2006,2,12343216,Distillates and Propane,Retail,Commercial,486,25923000.0,1.244,NaN
1,2006,3,12343216,Distillates and Propane,Retail,Commercial,486,45842000.0,1.239,NaN
2,2006,3,28952670,Distillates and Propane,Retail,Commercial,477,119000.0,2.028,NaN
3,2006,3,28952670,Distillates and Propane,Retail,Industrial,477,7000.0,2.028,NaN
4,2006,3,28952670,Distillates and Propane,Retail,Other,477,22000.0,2.030,NaN
...,...,...,...,...,...,...,...,...,...,...
3092,2008,5,967801416,Gasoline,Retail,Other,192,10000.0,3.983,NaN
3093,2008,5,967801416,Gasoline,Wholesale,Bulk,190,800000.0,3.749,NaN
3094,2008,5,967801416,Gasoline,Wholesale,Bulk,191,52000.0,3.804,NaN
3095,2008,5,967801416,Gasoline,Wholesale,Bulk,192,59000.0,3.913,NaN


In [27]:
sorted(df['Year'].unique())

[2006, 2007, 2008]

# 08

In [24]:
filename = "tbl_PIIRAWM08"
df = rawdata_to_df(filename)
df

,Reporting_Period,Report_End_Date,Product_Code,Receipts_Barrels,Inventory_Barrels,Report_Start_Date,Friday_Date,Company_Code,Terminal_ID,F10,F11,F12,F13
0,M,2023-06-30,163.0,380.0,62.0,2023-06-01 00:00:00,2023-06-01,8090938.0,63.0,NaN,NaN,NaN,0.0
1,M,2023-06-30,141.0,33.0,14.0,2023-06-01 00:00:00,2023-06-01,8090938.0,63.0,NaN,NaN,NaN,0.0
2,M,2023-06-30,478.0,26.0,14.0,2023-06-01 00:00:00,2023-06-01,8090938.0,63.0,NaN,NaN,NaN,0.0
3,M,2023-06-30,164.0,16.0,NaN,2023-06-01 00:00:00,2023-06-01,8090938.0,65.0,NaN,NaN,NaN,0.0
4,M,2023-06-30,163.0,30.0,53.0,2023-06-01 00:00:00,2023-06-01,8090938.0,65.0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145720,M,2024-08-31,141.0,55.0,33.0,2024-08-01 00:00:00,2024-08-01,8133480.0,30.0,NaN,NaN,NaN,0.0
145721,M,2024-08-31,478.0,26.0,38.0,2024-08-01 00:00:00,2024-08-01,8133480.0,30.0,NaN,NaN,NaN,0.0
145722,M,2024-08-31,489.0,187.0,83.0,2024-08-01 00:00:00,2024-08-01,8133480.0,30.0,NaN,NaN,NaN,0.0
145723,M,2024-08-31,217.0,360.0,222.0,2024-08-01 00:00:00,2024-08-01,8133480.0,30.0,NaN,NaN,NaN,0.0


# 810

In [14]:
filename = "tbl_PIIRAE810_NEW"
df = rawdata_to_df(filename)
df

,YEAR,MONTH,CID,RID,PRODUCT_CODE,ENTRY_FIELD,VALUE
0,2024,3,42137307.0,871205,411,D,855.00
1,2024,3,42137307.0,871205,465,D,720.00
2,2024,3,42137307.0,871205,466,D,135.00
3,2024,3,42137307.0,871205,511,D,18.00
4,2024,3,42137307.0,871205,510,D,18.00
...,...,...,...,...,...,...,...
853493,2023,10,8345464.0,457001,990,A,241.00
853494,2023,10,8345464.0,457001,399,A,8500.00
853495,2023,10,8345464.0,457001,401,A,8500.00
853496,2023,10,8345464.0,457001,40,A,1.74


In [19]:
sorted(df['YEAR'].unique())

[1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [23]:
filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)
df

,YEAR,MONTH,CID,RID,PRODUCT_CODE,ENTRY_FIELD,VALUE
0,2022,10,42137307,871205,481,A,220.0
1,2022,10,42137307,871205,180,C,11.0
2,2022,10,42137307,871205,163,D,1903.0
3,2022,10,42137307,871205,172,D,294.0
4,2022,10,42137307,871205,180,D,6.0
...,...,...,...,...,...,...,...
43225,2024,8,200003900,749007,175,E,24.0
43226,2024,8,200003900,749007,478,E,178.0
43227,2024,8,200003900,749007,163,G,375.0
43228,2024,8,200003900,749007,175,G,52.0


In [ ]:
filename = "tbl_PIIRAE810_NEW"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

df

filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

df = df[["YEAR", "MONTH", "RID", "PRODUCT_CODE", "ENTRY_FIELD", "VALUE"]]
df

df.loc[df.ENTRY_FIELD=="a", 'ENTRY_FIELD'] = "A"
df.loc[df.ENTRY_FIELD=="c", 'ENTRY_FIELD'] = "C"
df.loc[df.ENTRY_FIELD=="d", 'ENTRY_FIELD'] = "D"
df.loc[df.ENTRY_FIELD=="e", 'ENTRY_FIELD'] = "E"
df.loc[df.ENTRY_FIELD=="g", 'ENTRY_FIELD'] = "G"

df = df.pivot(index=["YEAR", "MONTH", "RID", "PRODUCT_CODE"], columns='ENTRY_FIELD', values='VALUE')
df = df.reset_index()
df

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Labels PRODUCT_CODE'] = df['PRODUCT_CODE'].map(dict_product_code)

df

# df.to_csv("CECM810.csv", index=False)
df_to_data(df, "CECM810.csv")



filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

df

df['RID'].unique()

dict_rid = {
    75005:	'Marathon Carson Wilmington',
    519010:	'PBF Torrance',
    533001:	'Kern Bakersfield',
    749007:	'PBF Martinez',
    810103:	'Chevron El Segundo',
    810104:	'Chevron Richmond',
    816106:	'Valero Benicia',
    897101:	'Valero Wilmington',
    871203:	'P66 Rodeo',
    871205:	'P66 Wilmington'
}

def get_df_rid(RID):
    y = df[df['RID']==RID].copy()
    return y        

dict_entry_field = {
    'A': 'Opening Inventory',
    'B': 'Receipts (Imports)',
    'C': 'Inputs',
    'D': 'Production',
    'E': 'Shipments (Exports)',
    'F': 'Fuel Use (Energy)',
    'G': 'Closing Inventory'   
}

# Production
# ENTRY_FIELD: D

# PRODUCT_CODE:
# Reformulated Blendstock for Oxygenate Blending (RBOB): 118
# Unfinished Oils, Total: 812
# Heavy Gas Oils: 840
# Distillate Fuel Oil, Total: 411
# Distillate Fuel Oil, 15ppm Sulfur and Under: 465
# Renewable Fuels, Total: 202
# Other Renewable Diesel Fuel (R100): 205
# Conventional Blendstock for Oxygenate Blending (CBOB): 139
# Petroleum Coke, Marketable: 21
# Still Gas: 45
# GTAB All Other Mogas Blending Components: 138
# Kerosene and Light Gas Oils: 830
# Other Renewable Fuels: 207

df_D = df[df['ENTRY_FIELD']=='D'].copy()

product_list = [118, 812, 840, 411, 465, 202, 205, 139, 21, 45, 138, 830, 207]

df_D = df_D[df_D['PRODUCT_CODE'].isin(product_list)]

# Inputs
# ENTRY_FIELD: C

# Crude Oil (incl. lease condensate), TOTAL	50
# Renewable Fuels, Total	202
# Other Renewable Diesel Fuel (R100)	205
# GTAB All Other Mogas Blending Components	138
# Inputs (Gain) or Production (Loss)	911
# Unfinished Oils, Total	812
# Naphthas and Lighter	820
# Residuum	850
# Distillate Fuel Oil, Total	411
# Distillate Fuel Oil, 15ppm Sulfur and Under	465
# Kerosene and Light Gas Oils	830
# Heavy Gas Oils	840
# Normal Butane/Butylene - Total	244
# Normal Butane -LRG	643

df_C = df[df['ENTRY_FIELD']=='C'].copy()

product_list = [55, 202, 205, 138, 911, 812, 820, 850, 411, 465, 830, 840, 244, 643]

df_C = df_C[df_C['PRODUCT_CODE'].isin(product_list)]

# Energy
# ENTRY_FIELD: F

# Still Gas	45
# Petroleum Coke, Catalyst	22

df_F = df[df['ENTRY_FIELD']=='F'].copy()

product_list = [45, 22]

df_F = df_F[df_F['PRODUCT_CODE'].isin(product_list)]

df = pd.concat([df_D,df_C,df_F], axis='rows')

df['VAR'] = df['ENTRY_FIELD'].astype(str) + df['PRODUCT_CODE'].astype(str)

df = df[['YEAR', 'MONTH', 'RID', 'VAR', 'VALUE']]
df

df = df.pivot(index=['YEAR', 'MONTH', 'RID'], columns='VAR', values='VALUE') 
df

df = df.reset_index()

# df.to_csv("E810.csv", index=False)
df_to_data(df, "E810.csv")